In [15]:
import pandas as pd
import urllib.request
import json
import time

# Define the API Key and Base URL
API_KEY = '***REDACTED***'
BASE_URL = 'https://api.stlouisfed.org/fred/series/observations'

# Function to fetch PCPI for each county and year
def fetch_pcpi(series_id, year):
    # Construct the query URL for the PCPI series
    query_url = (
        f"{BASE_URL}?"
        f"series_id={series_id}&"
        f"api_key={API_KEY}&"
        f"file_type=json"
    )
    try:
        response = urllib.request.urlopen(query_url)
        response_data = json.loads(response.read())
        
        # Extract the value for the specified year
        if 'observations' in response_data:
            for observation in response_data['observations']:
                if observation['date'].startswith(year):
                    return float(observation['value'])
    except urllib.error.HTTPError as e:
        if e.code == 429:
            print(f"Rate limit hit. Retrying after delay...")
            time.sleep(5)  # Delay for 5 seconds
            return fetch_pcpi(series_id, year)  # Retry
        else:
            print(f"HTTP Error for {series_id}, year {year}: {e}")
    except Exception as e:
        print(f"Error fetching data for {series_id}, year {year}: {e}")
    return None

# Initial request to get county series IDs
COUNTY_BASE_URL = 'https://api.stlouisfed.org/fred/release/tables'
params = {
    'release_id': 175,
    'element_id': 266305,  # Ensure this is correct for California
    'api_key': API_KEY,
    'file_type': 'json',
    'od': '2022-01-01'  # Observation date for 2022
}
query_url = (
    f"{COUNTY_BASE_URL}?"
    f"release_id={params['release_id']}&"
    f"element_id={params['element_id']}&"
    f"api_key={params['api_key']}&"
    f"file_type={params['file_type']}&"
    f"od={params['od']}"
)

# Make the API request to get county data
try:
    response = urllib.request.urlopen(query_url)
    response_data = json.loads(response.read())
    
    # Extract the 'elements' key from the response
    if 'elements' in response_data:
        elements = response_data['elements']
        
        # Prepare data
        data = []
        years = [str(year) for year in range(2011, 2023)]  # Years 2011–2022
        for key, value in elements.items():
            county_name = value['name']
            series_id = value['series_id']
            
            # Fetch PCPI for each year
            for year in years:
                pcpi = fetch_pcpi(series_id, year)
                if pcpi is not None:
                    data.append({'County Name': county_name, 'Year': year, 'PCPI': pcpi})
                time.sleep(0.5)  # Delay of 0.5 seconds between requests
        
        # Create a DataFrame
        df = pd.DataFrame(data)
        print("DataFrame:")
        print(df.head())  # Display the resulting DataFrame
        
        # Optionally, save the DataFrame as a CSV file
        df.to_csv('PCPI_California_2011_2022.csv', index=False)
    else:
        print("No 'elements' key found in the response. Check the response structure.")
except Exception as e:
    print(f"An error occurred: {e}")


DataFrame:
      County Name  Year     PCPI
0  Alameda County  2011  50432.0
1  Alameda County  2012  52296.0
2  Alameda County  2013  54131.0
3  Alameda County  2014  57009.0
4  Alameda County  2015  61618.0


In [17]:
df.head(5)

,County Name,Year,PCPI
0,Alameda County,2011,50432.0
1,Alameda County,2012,52296.0
2,Alameda County,2013,54131.0
3,Alameda County,2014,57009.0
4,Alameda County,2015,61618.0
